In [2]:
import os
import glob
import datetime
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('max_rows', 300)
pd.set_option('max_columns', 200)

perc = [0.01, 0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99]

startDate = '20201207'
endDate = '20201216'

readPath = r'L:\orderLog\data\***'
dataPathLs = np.array(glob.glob(readPath))
dateLs = np.array([os.path.basename(i).split('.')[0] for i in dataPathLs])
dataPathLs = dataPathLs[(dateLs >= startDate) & (dateLs <= endDate)]
print(dataPathLs)

rawOrderLog = []
for thisPath in dataPathLs:
    data = pickle.load(open(thisPath, 'rb'))
    data = data.rename(columns={'mdClockAtArrival': 'caamd'})
    rawOrderLog += [data]
rawOrderLog = pd.concat(rawOrderLog, sort=False)

for col in ['clockAtArrival', 'caamd', 'secid', 'updateType', 'vai', 'absFilledThisUpdate', 'orderDirection', 'absOrderSize',
            'absOrderSizeCumFilled', 'date', 'accCode', 'mse']:
    rawOrderLog[col] = rawOrderLog[col].fillna(0).astype('int64')   
rawOrderLog = rawOrderLog.sort_values(by=['date', 'secid', 'vai', 'accCode', 'clockAtArrival']).reset_index(drop=True)

rawOrderLog = rawOrderLog[rawOrderLog["secid"] >= 1000000]

if rawOrderLog[rawOrderLog.duplicated(['date', 'secid', 'vai', 'accCode', 'clockAtArrival', 'updateType', \
                                    'orderDirection', 'absOrderSize'], keep=False)].shape[0] != 0:
    display('There are accounts with duplicated ticks:')
    display(rawOrderLog[rawOrderLog.duplicated(['date', 'secid', 'vai', 'accCode', 'clockAtArrival', 'updateType', \
                                    'orderDirection', 'absOrderSize'], keep=False)]\
.groupby(['date', 'colo', 'accCode'])['ars'].size())
    rawOrderLog = rawOrderLog.drop_duplicates(['date', 'secid', 'vai', 'accCode', 'clockAtArrival', 'updateType', \
                                    'orderDirection', 'absOrderSize'], keep='first')

display('There are ticks with orderDirection 0')
display(rawOrderLog[rawOrderLog['orderDirection'] == 0][['date', 'colo', 'accCode', \
            'secid', 'vai', 'updateType', 'sdd', 'orderDirection', 'absOrderSize', 'internalId', 'orderId']])

assert(rawOrderLog[rawOrderLog['updateType'] == 0][rawOrderLog[rawOrderLog['updateType'] == 0]\
                                                   .duplicated(['date', 'colo', 'accCode', 'secid', 'orderDirection',
                                                                'vai', 'absOrderSize', 'internalId'], keep=False)].shape[0] == 0)
try:
    assert(rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)][rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)]\
                                                       .duplicated(['date', 'colo', 'accCode', 'secid', 'orderDirection',
                                                                    'absOrderSize', 'internalId'], keep=False)].shape[0] == 0)
except:
    print('There are orders with all things same except sdd')
    print(rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)][rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)]\
                                                       .duplicated(['date', 'colo', 'accCode', 'secid', 'orderDirection',
                                                                    'absOrderSize', 'internalId'], keep=False)])
    assert(rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)][rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)]\
                                                       .duplicated(['date', 'colo', 'accCode', 'secid', 'orderDirection',
                                                                    'absOrderSize', 'internalId', 'sdd'], keep=False)].shape[0] == 0)
try:
    assert(sum(rawOrderLog[(rawOrderLog['updateType'] != 0) & (rawOrderLog['accCode'] != 8856)].groupby(['date', 'colo', 'accCode', 'secid', 
                'orderDirection', 'absOrderSize', 'internalId'])['orderId'].nunique() != 1) == 0) 
except:
    print('There are orders with same internalId but different orderId other than accCode 8856 case')
    print(rawOrderLog[(rawOrderLog['updateType'] != 0) & (rawOrderLog['accCode'] != 8856)].groupby(['date', 'colo', 'accCode', 'secid', 
                'orderDirection', 'absOrderSize', 'internalId'])['orderId'].nunique()[rawOrderLog[(rawOrderLog['updateType'] != 0) & (rawOrderLog['accCode'] != 8856)].groupby(['date', 'colo', 'accCode', 'secid', 
                'orderDirection', 'absOrderSize', 'internalId'])['orderId'].nunique() > 1])

r2 = rawOrderLog[(rawOrderLog['accCode'] != 8856) & (rawOrderLog['orderDirection'] != 0)]
r1 = rawOrderLog[(rawOrderLog['accCode'] == 8856) & (rawOrderLog['orderDirection'] != 0)]
r1['test'] = r1.groupby(['date', 'colo', 'accCode', 'secid', 
            'orderDirection', 'absOrderSize']).grouper.group_info[0]
r1 = r1.sort_values(by=['test', 'clockAtArrival'])
r1.loc[r1['updateType'] != 0, 'vai'] = np.nan
r1['vai'] = r1.groupby('test')['vai'].ffill()
r2['test'] = r2.groupby(['date', 'colo', 'accCode', 'secid', 
            'orderDirection', 'absOrderSize', 'internalId']).grouper.group_info[0]
r2 = r2.sort_values(by=['test', 'clockAtArrival'])
r2.loc[r2['updateType'] != 0, 'vai'] = np.nan
r2['vai'] = r2.groupby('test')['vai'].ffill()
assert(sum(r1[r1['updateType'] != 0].groupby(['test', 'vai'])['orderId'].nunique() != 1) == 0)
try:
    assert(sum(r2[r2['updateType'] != 0].groupby(['test', 'vai'])['orderId'].nunique() != 1) == 0)
except:
    a = r2[r2['updateType'] != 0].groupby(['test', 'vai'])['orderId'].nunique()[r2[r2['updateType'] != 0].groupby(['test', 'vai'])['orderId'].nunique() != 1].reset_index()
    print(pd.merge(r2, a[['test', 'vai']], on=['test', 'vai'], how='inner')[['secid', 'accCode', 'colo', 'vai', 'updateType', 'sdd', 'internalId', 'orderId', 'absOrderSize', 'absFilledThisUpdate', 'absOrderSizeCumFilled', 'orderPrice', 'tradePrice']])
orderLog = pd.concat([r1, r2])
del r1
del r2    

orderLog = orderLog.sort_values(by=['date', 'colo', 'accCode', 'secid', 'vai', 'clockAtArrival']).reset_index(drop=True)
orderLog['clock'] = orderLog['clockAtArrival'].apply(lambda x: datetime.datetime.fromtimestamp(x/1e6))
orderLog['broker'] = orderLog['accCode'] // 100
orderLog["broker"] = np.where(orderLog["accCode"].astype(str).apply(lambda x: len(x) == 6), orderLog['accCode'] // 10000, orderLog["broker"])
orderLog['colo_broker'] = orderLog['colo'].str[:2] + '_' + orderLog['broker'].astype('str')
orderLog['order'] = orderLog.groupby(['date', 'colo', 'accCode', 'secid', 'vai']).grouper.group_info[0]
orderLog['group'] = orderLog.groupby(['date', 'secid', 'vai']).grouper.group_info[0]
orderLog['startClock'] = orderLog.groupby(['order'])['clockAtArrival'].transform('first')
orderLog['duration'] = orderLog['clockAtArrival'] - orderLog['startClock']
orderLog['orderPrice'] = orderLog['orderPrice'].apply(lambda x: round(x, 2))
orderLog['tradePrice'] = orderLog['tradePrice'].apply(lambda x: round(x, 2))
orderLog['orderDirection1'] = np.where(orderLog["orderDirection"] == -2, -1, np.where(
    orderLog["orderDirection"] == 2, 1, orderLog["orderDirection"]))
orderLog["ars"] = orderLog.groupby(['order'])['ars'].transform('first')


### Assertion 1:  make sure same direction in same date, secid, vai
print('=======================================================================================')
print('1. same date, secid, vai: same direction')
orderLog['directNum'] = orderLog.groupby(['date', 'secid', 'vai'])['orderDirection1'].transform('nunique')
if len(orderLog[orderLog['directNum'] != 1]) > 0:
    print('opposite direction for same date, same secid, same vai')
    display(orderLog[(orderLog['directNum'] != 1) & (orderLog['updateType'] == 0)].groupby(['accCode'])['orderDirection'].size())
    orderLog = orderLog[orderLog['directNum'] == 1]

assert((orderLog.groupby(['date', 'secid', 'vai'])['orderDirection1'].nunique() == 1).all() == True)

## Assertion 2:  make sure each account, secid, vai only has one insertion
print('=======================================================================================')
print('2. same date, secid, vai, accCode: one insertion')
a = orderLog[orderLog['updateType'] == 0].groupby(['date', 'colo', 'accCode', 'secid', 'vai', 'order'])['clockAtArrival'].count()
if len(a[a > 1]) > 0:
    print('more than one insertion at same time')
    a = a[a>1].reset_index()
    display(a)
    display(a.groupby(['accCode'])['date'].size())
    orderLog = orderLog[~(orderLog['order'].isin(a['order'].unique()))]

orderLog['isMsg'] = np.where(orderLog['updateType'] == 0, 
                             np.where(orderLog['mse'] == 100, 1, 0), np.nan)
orderLog['isMsg'] = orderLog.groupby(['order'])['isMsg'].ffill()

placeSZE = orderLog[(orderLog['secid'] >= 2000000) & (orderLog['updateType'] == 0)]
print('%.2f%% SZE orders triggered by msg data'%(placeSZE[placeSZE['isMsg'] == 1].shape[0]/placeSZE.shape[0]*100))


### Assertion 3:  check IPO stocks selling status
print('=======================================================================================')
print('3. IPO stocks selling (ars = 301, 302)')
if orderLog[orderLog['ars'].isin([301, 302])].shape[0] != 0:
    kk = orderLog[orderLog['ars'].isin([301, 302])]
    print(kk)
    try:
        assert(kk[kk['orderDirection1'] == 1].shape[0] == 0)
        print('we only sell, never buy')
    except:
        print('There are IPO buy side orders!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
        print(kk[kk['orderDirection1'] == 1])
    kk1 = kk[kk['updateType'] == 0]
    kk1 = kk1.sort_values(by=['accCode', 'secid','clockAtArrival'])
    kk1['diff'] = kk1.groupby(['accCode', 'secid'])['clockAtArrival'].apply(lambda x: x-x.shift(1))
    kk1['diff'] = kk1['diff'].fillna(0)
    try:
        assert(kk1[kk1['diff'] < 10e6].shape[0] == 0)
        print('for each stock in the same account, there is no insertion within 10 seconds of the previous insertion')
    except:
        print('There are insertion within 10 seconds for orders under same account same stock!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
        print(kk1[kk1['diff'] < 10e6])
    kk2 = kk[(kk['updateType'] == 1)]
    try:
        assert(kk2[kk2['duration'] < 3e6].shape[0] == 0)
        print('for each stock in the same account, the cancellation of an order happens more than 3 seconds after the insertion')
    except:
        print('There are cancellation within 3 seconds for orders under same account same stock!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
        print(kk2[kk2['duration'] < 3e6])


### Assertion 4: check updateType == 7 orders, make sure updateType == 7 orders < 20 per account, < 100 in total
print('=======================================================================================')
print('4. updateType 7 orders')
if orderLog[orderLog['updateType'] == 7].shape[0] != 0:
    assert(orderLog[orderLog['updateType'] == 7].groupby(['date', 'accCode'])['order'].nunique().max() < 20)
    assert(orderLog[orderLog['updateType'] == 7].groupby(['date', 'accCode'])['order'].nunique().sum() < 100)

### Assertion 5: check updateType == 6 orders, make sure updateType == 6 orders < 5% per account
print('=======================================================================================')
print('5. updateType 6 orders')
k1 = orderLog[orderLog['updateType'] == 6].groupby(['date', 'accCode'])['order'].nunique().reset_index()
k2 = orderLog.groupby(['date', 'accCode'])['order'].nunique().reset_index()
k = pd.merge(k1, k2, on=['date', 'accCode'], how='left')
k['prob'] = k['order_x']/k['order_y']
try:
    assert(sum(k['prob'] >= 0.05) == 0)
except:
    print('There are accounts with more than 5% updateType 6 orders!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
    print(k[k['prob'] >= 0.05])

### Assertion 6: check CYB orders, make sure all CYB stocks have absOrderSize < 30w
print('=======================================================================================')
print('6. CYB stocks order size < 30w')
try:
    cyb = orderLog[(orderLog['secid'] >= 2300000) & (orderLog['updateType'] == 0)].groupby('date')['absOrderSize'].max().reset_index()
    assert(cyb[cyb['absOrderSize'] > 300000].shape[0] == 0)
except:
    print('CYB stocks total absOrderSize >= 30w!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
    
     
### Assertion 7:  make sure there is no unexpected updateType 
print('=======================================================================================')
print('7. unexpected updateType')
def getTuple(x):
    return tuple(i for i in x)

checkLog = orderLog[~((orderLog['updateType'] == 4) & (orderLog.groupby(['order'])['updateType'].shift(-1) == 4))]
checkLog = checkLog.groupby(['order'])['updateType'].apply(lambda x: getTuple(x)).reset_index()
checkLog['status'] = np.where(checkLog['updateType'].isin([(0, 2, 4), (0, 2, 1, 4), (0, 2, 1, 2, 4), (0, 2, 4, 1, 4), (0, 4), (0, 1, 4), (0, 4, 1, 4), (0, 2, 2, 4), (0, 4, 2, 4), (0, 2, 2, 1, 4), (0, 2, 2, 4, 1, 4)]),0,
                     np.where(checkLog['updateType'].isin([(0, 2, 4, 1, 3), (0, 2, 4, 1, 4, 3), (0, 2, 1, 4, 3), (0, 4, 1, 3), (0, 1, 4, 3),
                                                               (0, 2, 2, 4, 1, 3), (0, 2, 2, 4, 1, 4, 3), (0, 2, 2, 1, 4, 3), (0, 4, 2, 4, 1, 3),
                                                               (0, 4, 2, 1, 3), (0, 4, 1, 4, 3), (0, 4, 1)]), 1,
                     np.where(checkLog['updateType'].isin([(0, 2, 1, 3), (0, 2, 2, 1, 3), (0, 2, 3), (0, 3), (0, 1, 3), (0, ), (0, 2), (0, 2, 1), (0, 2, 2)]), 2, 3)))

orderLog = pd.merge(orderLog, checkLog[['order', 'status']], how='left', on=['order'], validate='many_to_one')
orderLog = orderLog[orderLog['status'].isin([0, 1, 2])].reset_index(drop=True)

### Assertion 8:  make sure status==0 got all traded
print('=======================================================================================')
print('8. status == 0: all traded')
a = orderLog[orderLog['status'] == 0]
a = a.groupby(['order'])[['absOrderSizeCumFilled', 'absOrderSize']].max().reset_index()
a.columns = ['order', 'filled', 'total']
print('in total trade, any fill != total cases')
display(a[a['filled'] != a['total']])
if a[a['filled'] != a['total']].shape[0] > 0:
    removeOrderLs = a[a['filled'] != a['total']]['order'].unique()
    orderLog = orderLog[~(orderLog['order'].isin(removeOrderLs))]
    
### Assertion 9:  make sure status==1 got partial traded
print('=======================================================================================')
print('9. status == 1: partial traded')
a = orderLog[orderLog['status'] == 1]
a = a.groupby(['order'])[['absOrderSizeCumFilled', 'absOrderSize']].max().reset_index()
a.columns = ['order', 'filled', 'total']
print('in partial trade, any fill >= total or fill is 0 cases for updateType 4')
display(a[(a['filled'] >= a['total']) | (a['filled'] == 0)])
if a[(a['filled'] >= a['total']) | (a['filled'] == 0)].shape[0] > 0:
    removeOrderLs = a[(a['filled'] >= a['total']) | (a['filled'] == 0)]['order'].unique()
    orderLog = orderLog[~(orderLog['order'].isin(removeOrderLs))]
    
### Assertion 10: make sure no cancellation within 1 sec
print('=======================================================================================')
print('10. no cancellation within 1 sec')
a = orderLog[(orderLog['updateType'] == 1) & (orderLog['duration'] < 1e6)]
print('any cancellation within 1 sec')
display(a)
if a.shape[0] > 0:
    removeOrderLs = a['order'].unique()
    orderLog = orderLog[~(orderLog['order'].isin(removeOrderLs))]


### Assertion 11: make sure no order has shares > 80w or notional > 800w
print('=======================================================================================')
print('11. Orders with size > 80w or notional > 800w')
orderLog['orderNtl'] = orderLog['absOrderSize'] * orderLog['orderPrice']
if orderLog[orderLog['absOrderSize'] > 800000].shape[0] > 0:
    print('some order quantity are > 80w')
    print(orderLog[orderLog['absOrderSize'] > 800000].groupby(['colo', 'accCode'])['order'].nunique())
    display(orderLog[orderLog['absOrderSize'] > 800000][['date', 'accCode', 'secid', 'vai', 'absOrderSize', 'orderPrice',
                                                         'orderNtl', 'orderDirection', 'clock', 'order']])
            
if orderLog[orderLog['orderNtl'] > 8000000].shape[0] > 0:
    print('some order ntl are > 800w')
    print(orderLog[orderLog['orderNtl'] > 8000000].groupby(['colo', 'accCode'])['order'].nunique())
    display(orderLog[orderLog['orderNtl'] > 8000000][['date', 'accCode', 'secid', 'vai', 'absOrderSize', 'orderPrice',
                                                      'orderNtl', 'orderDirection', 'clock', 'order', "updateType", 
                                                      "tradePrice", "absOrderSizeCumFilled", "absFilledThisUpdate"]])

removeOrderLs = list(set(orderLog[orderLog['absOrderSize'] > 800000]['order'].unique()) | set(orderLog[orderLog['orderNtl'] > 8000000]['order'].unique()))
orderLog = orderLog[~(orderLog['order'].isin(removeOrderLs))]


orderLog = orderLog.sort_values(by=['date', 'secid', 'vai', 'accCode', 'clockAtArrival']).reset_index(drop=True)

orderLog["mrsb90"] = orderLog.groupby(['order'])['mrsb90'].transform('first')
orderLog["mrss90"] = orderLog.groupby(['order'])['mrss90'].transform('first')
orderLog["mrstaat"] = orderLog.groupby(['order'])['mrstaat'].transform('first')
orderLog["mrstauc"] = orderLog.groupby(['order'])['mrstauc'].transform('first')
orderLog["aaa"] = orderLog.groupby(['order'])['aaa'].transform('first')

orderLog['m1'] = orderLog['mrstaat'].apply(lambda x: x - (x // 10000) * 10000)
orderLog['m2'] = orderLog['mrstauc'].apply(lambda x: x - (x // 10000) * 10000)
if orderLog[orderLog['mrsb90'] == '-'].shape[0] != 0:
    display(orderLog[orderLog['mrsb90'] == '-'])
orderLog = orderLog[orderLog['mrsb90'] != '-']
orderLog['mrsb90'] = orderLog['mrsb90'].astype(float)
if orderLog[orderLog['aaa'] == '-'].shape[0] != 0:
    display(orderLog[orderLog['aaa'] == '-'])
orderLog = orderLog[orderLog['aaa'] != '-']
orderLog['aaa'] = orderLog['aaa'].astype(float)
orderLog.loc[(orderLog['orderDirection'] >= 1) &\
         (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrsb90']) < 1e-12), 'mrstauc'] = \
orderLog.loc[(orderLog['orderDirection'] >= 1) &\
         (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrsb90']) < 1e-12), 'm2']

orderLog.loc[(orderLog['orderDirection'] >= 1) &\
         (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrsb90']) < 1e-12), 'mrstaat'] = \
orderLog.loc[(orderLog['orderDirection'] >= 1) &\
         (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrsb90']) < 1e-12), 'm1']

orderLog.loc[(orderLog['orderDirection'] < 1) &\
         (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrss90']) < 1e-12), 'mrstauc'] = \
orderLog.loc[(orderLog['orderDirection'] < 1) &\
         (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrss90']) < 1e-12), 'm2']

orderLog.loc[(orderLog['orderDirection'] < 1) &\
         (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrss90']) < 1e-12), 'mrstaat'] = \
orderLog.loc[(orderLog['orderDirection'] < 1) &\
         (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrss90']) < 1e-12), 'm1']    

orderLog['exchange'] = np.where(orderLog['secid'] >= 2000000, 'SZE', 'SSE')
orderLog['orderNtl'] = orderLog['orderPrice'] * orderLog['absOrderSize']
orderLog['tradeNtl'] = np.where(orderLog['updateType'] == 4, orderLog['tradePrice']*orderLog['absFilledThisUpdate'], 0)
orderLog["mrstaat"] = orderLog.groupby(['order'])['mrstaat'].transform('first')
orderLog["ars"] = orderLog.groupby(['order'])['ars'].transform('first')
orderLog["mrstauc"] = orderLog.groupby(['order'])['mrstauc'].transform('first')
orderLog = orderLog[~orderLog['ars'].isnull()]
orderLog['sta'] = np.where(orderLog['mrstaat'] == 1000, '1. staone', np.where(
orderLog['mrstaat'] == 3000, '2. statwo', np.where(
orderLog['mrstaat'].isin([11000, 13000]), '3. sta300', 'else')))
display(orderLog[orderLog['sta'] == 'else'][['date', 'secid', 'colo', 'accCode', 'updateType', 'vai', 'absOrderSize', 'orderDirection', 'mrstaat', 'mrstauc', 'sdd', 'ApplSeqNum', 'sta']])
orderLog = orderLog[orderLog['mrstaat'].isin([11000, 13000, 1000, 3000])]
display(orderLog[orderLog['updateType'] == 0].groupby(['mrstaat', 'mrstauc'])['date'].size())

['L:\\orderLog\\data\\20201214.pkl' 'L:\\orderLog\\data\\20201207.pkl'
 'L:\\orderLog\\data\\20201209.pkl' 'L:\\orderLog\\data\\20201208.pkl'
 'L:\\orderLog\\data\\20201210.pkl' 'L:\\orderLog\\data\\20201211.pkl'
 'L:\\orderLog\\data\\20201215.pkl' 'L:\\orderLog\\data\\20201216.pkl']


'There are ticks with orderDirection 0'

,date,colo,accCode,secid,vai,updateType,sdd,orderDirection,absOrderSize,internalId,orderId
251006,20201207,zs_54_01,5474,2000565,-1,1,52618.0,0,0,-1.0,8.689164e+08
271922,20201207,zs_96_08,9685,2000802,-1,7,40338.0,0,0,-1.0,-1.000000e+00
316660,20201207,zs_66_01,6634,2002177,-1,1,50844.0,0,0,-1.0,1.812809e+10
350323,20201207,zs_52_08,5281,2002410,-1,1,34461.0,0,0,-1.0,1.105800e+05
369671,20201207,zs_52_06,5269,2002553,-1,1,35149.0,0,0,-1.0,2.627480e+05
405701,20201207,zs_66_01,6634,2002753,-1,1,51509.0,0,0,-1.0,1.812809e+10
415422,20201207,zs_52_06,5269,2002808,-1,1,34995.0,0,0,-1.0,2.344340e+05
473560,20201207,zs_52_06,5269,2300175,-1,1,40457.0,0,0,-1.0,8.723280e+05
495446,20201207,zs_52_09,5291,2300296,-1,1,39960.0,0,0,-1.0,8.262520e+05
528555,20201207,zs_94_05,9454,2300468,-1,7,37155.0,0,0,-1.0,-1.000000e+00


There are orders with all things same except sdd
         ApplSeqNum       aaa  absFilledThisUpdate  absOrderSize  \
5137081   3686264.0   0.00044                    0           100   
5137362  18252604.0  0.001406                    0           100   
5169063   3688220.0  0.000419                    0           100   
5169086  14404537.0  0.000657                    0           100   

         absOrderSizeCumFilled  accCode  ars  caamd  cancellationPending  \
5137081                      0   966301  0.0      0                  0.0   
5137362                      0   966301  0.0      0                  0.0   
5169063                      0   966301  0.0      0                  0.0   
5169086                      0   966301  0.0      0                  0.0   

               cfe         clock    clockAtArrival  cumSharesBought  \
5137081  506777.82  1.608083e+15  1608083268955951            100.0   
5137362  147834.78  1.608101e+15  1608101490951270              0.0   
5169063  568630.

C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


...

10. no cancellation within 1 sec
any cancellation within 1 sec


,ApplSeqNum,aaa,absFilledThisUpdate,absOrderSize,absOrderSizeCumFilled,accCode,ars,caamd,cancellationPending,cfe,clock,clockAtArrival,cumSharesBought,cumSharesBuyInserted,cumSharesSellInserted,cumSharesSold,date,finalState,gfe,hee,insertedShortOrder,insertionPending,internalId,inv_L,inv_L0,inv_S,inv_S0,l4algoDebug,l4tr,locateShares,locateSharesTotal,mfe,mra100,mrb100,mrm,mrm25,mrmum,mrrlma,mrsb300,mrsb90,mrss300,mrss90,mrstaat,mrstauc,mrstaum,mrv,ms,mse,mt,mta,mv,orderDirection,orderId,orderOutstanding,orderPrice,orderSysId,sdd,secid,sequenceNo,session,threadId,totalActions,totalCanceled,tradeId,tradePrice,underlyingIndex,updateType,vai,zipFile,colo,caa_orderLog,start_time,Price,OrderQty,Side,statusLs,TradePriceLs,TradeQtyLs,beta_60,adjMid_F30s,adjMid_F90s,adjMid_F300s,indexClose,indexClose_F30s,indexClose_F90s,indexClose_F300s,test,broker,colo_broker,order,group,startClock,duration,orderDirection1,directNum,isMsg,status


11. Orders with size > 80w or notional > 800w


,date,secid,colo,accCode,updateType,vai,absOrderSize,orderDirection,mrstaat,mrstauc,sdd,ApplSeqNum,sta
126894,20201207,1601686,zt_88_06,8854,0,302.0,735,-1,0.0,0.0,134146000.0,0.0,else
126895,20201207,1601686,zt_88_06,8854,2,302.0,735,-1,0.0,0.0,-1.0,0.0,else
126896,20201207,1601686,zt_88_06,8854,4,302.0,735,-1,0.0,0.0,-1.0,0.0,else
126897,20201207,1601686,zt_88_06,8886,0,302.0,735,-1,0.0,0.0,134146000.0,0.0,else
126898,20201207,1601686,zt_88_06,8886,2,302.0,735,-1,0.0,0.0,-1.0,0.0,else
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5374435,20201216,2300595,zt_96_01,966301,1,1490268.0,100,1,0.0,0.0,95950410.0,0.0,else
5374436,20201216,2300595,zt_96_01,966301,3,1490268.0,100,1,0.0,0.0,-1.0,0.0,else
5375327,20201216,2300595,zt_96_01,966301,0,4393631.0,100,1,0.0,0.0,135327880.0,15075042.0,else
5375328,20201216,2300595,zt_96_01,966301,2,4393631.0,100,1,0.0,0.0,-1.0,0.0,else


mrstaat  mrstauc
1000.0   0.0        189889
3000.0   1000.0     921887
         2000.0      62527
         3000.0      23310
11000.0  10000.0      7156
         20000.0       538
         30000.0       367
13000.0  11000.0    149080
         12000.0      5595
         13000.0      1333
         21000.0      3517
         22000.0      4314
         23000.0       891
         31000.0      1681
         32000.0      1515
         33000.0      1928
Name: date, dtype: int64

In [6]:
orderLog = orderLog[orderLog['ars']%10 == 1]
orderLog['server_account'] = orderLog['colo'].apply(lambda x: x.replace('_', '')) + '_' + orderLog['accCode'].astype(str)
orderLog[(orderLog['sta'].isin(['3. sta300'])) & (orderLog['updateType'] == 0)].groupby(['date', 'server_account'])['secid'].nunique()

date      server_account
20201207  zs8804_8924       194
          zs8804_892402       1
          zs8804_8967       178
          zs8804_896702     131
          zs9606_9756       508
          zs9606_975601     273
          zs9606_9765       206
          zt5205_537403      79
          zt8806_8943       153
          zt9406_9551       145
20201208  zs8804_8924       162
          zs8804_8967       194
          zs8804_896702     111
          zs9606_9756       432
          zs9606_975601     242
          zs9606_9765       158
          zt5205_537403      74
          zt8806_8943       128
          zt9406_9551        99
20201209  zs8804_8924       253
          zs8804_8967        88
          zs8804_896702     172
          zs9606_9756       679
          zs9606_975601     356
          zs9606_9765       176
          zt5205_537403     121
          zt8806_8854         1
          zt8806_8943       139
          zt9406_9551       176
20201210  zs8804_8924       203
          zs880

In [7]:
import pymongo
import io
import pandas as pd
import pickle
import datetime
import time
import gzip
import lzma
import pytz
import pyarrow as pa
import pyarrow.parquet as pq
import numpy as np
def DB(host, db_name, user, passwd):
    auth_db = db_name if user not in ('admin', 'root') else 'admin'
    url = 'mongodb://%s:%s@%s/?authSource=%s' % (user, passwd, host, auth_db)
    client = pymongo.MongoClient(url, maxPoolSize=None)
    db = client[db_name]
    return db

def read_memb_daily(db, name, start_date=None, end_date=None, skey=None, index_id=None, interval=None, col=None, return_sdi=True):
    collection = db[name]
    # Build projection
    prj = {'_id': 0}
    if col is not None:
        if return_sdi:
            col = ['skey', 'date', 'interval'] + col
        for col_name in col:
            prj[col_name] = 1

    # Build query
    query = {}
    if skey is not None:
        query['skey'] = {'$in': skey}
    if index_id is not None:
        query['index_id'] = {'$in': index_id}
    if interval is not None:
        query['interval'] = {'$in': interval}
    if start_date is not None:
        if end_date is not None:
            query['date'] = {'$gte': start_date, '$lte': end_date}
        else:
            query['date'] = {'$gte': start_date}
    elif end_date is not None:
        query['date'] = {'$lte': end_date}

    # Load data
    cur = collection.find(query, prj)
    df = pd.DataFrame.from_records(cur)
    if df.empty:
        df = pd.DataFrame()
    else:
        df = df.sort_values(by=['date', 'index_id', 'skey'])
    return df    

database_name = 'com_md_eq_cn'
user = "zhenyuy"
password = "bnONBrzSMGoE"

pd.set_option('max_columns', 200)
db1 = DB("192.168.10.178", database_name, user, password)
memb1 = read_memb_daily(db1, 'index_memb', 20201211, 20201211)

In [8]:
checkLog1 = pd.concat([orderLog[(orderLog['server_account'].isin(['zs6601_6623', 'zs6601_6627', 'zs6601_6683', 'zt8803_897002'])) & (orderLog['date'] < 20201214)],
                       orderLog[(orderLog['server_account'].isin(['zt5205_537501', 'zs5206_5384', 'zt8806_8886', 'zs9608_974101', 'zs9608_9741'])) & (orderLog['date'] < 20201215)]])
checkLog2 = pd.concat([orderLog[(orderLog['server_account'].isin(['zs6601_6623', 'zs6601_6627', 'zs6601_6683', 'zt8803_897002'])) & (orderLog['date'] >= 20201214) & (orderLog['sta'] == '3. sta300')],
                       orderLog[(orderLog['server_account'].isin(['zt5205_537501', 'zs5206_5384', 'zt8806_8886', 'zs9608_974101', 'zs9608_9741'])) & (orderLog['date'] >= 20201215) & (orderLog['sta'] == '3. sta300')]])
checkLog1['tag'] = 'before rollout'
checkLog2['tag'] = 'after rollout'
checkLog = pd.concat([checkLog1, checkLog2])
checkLog = checkLog[(checkLog["updateType"] == 0)]
checkLog['index'] = np.where(checkLog['server_account'].isin(['zt5205_537501', 'zs5206_5384']), 'IF', np.where(
checkLog['server_account'].isin(['zt8803_897002', 'zt8806_8886']), 'IC', np.where(
checkLog['server_account'].isin(['zs6601_6623', 'zs6601_6627', 'zs6601_6683']), 'CSI1000', 'CSIRest')))
# display(checkLog.groupby(['date', 'tag', 'index'])['server_account'].unique())
checkLog = checkLog[checkLog['caamd'] != 0]
checkLog['nn'] = checkLog.groupby(['exchange', 'server_account', 'secid'])['tag'].transform('nunique')
checkLog = checkLog[checkLog['nn'] == 2]
checkLog['broker'] = checkLog['colo'].str[:2] + '_' + checkLog['accCode'].astype(str).str[:2]
checkLog['internal_latency'] = checkLog["clockAtArrival"] - checkLog["caamd"]
re = checkLog.groupby(['tag', 'index', 'broker'])['internal_latency'].describe().reset_index()
re = re[['tag', 'index', 'broker', 'count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max']]
re = re.fillna(0)
for i in re.columns[(re.columns != 'tag') & (re.columns != 'broker') & (re.columns != 'index')]:
    re[i] = re[i].astype('int64')
    
from IPython.display import display, HTML
HTML(re.groupby(['index', 'broker', 'tag'])[['count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max']].first().to_html())

In [9]:
checkLog1 = pd.concat([orderLog[(orderLog['server_account'].isin(['zs6601_6623', 'zs6601_6627', 'zs6601_6683', 'zt8803_897002'])) & (orderLog['date'] < 20201214)],
                       orderLog[(orderLog['server_account'].isin(['zt5205_537501', 'zs5206_5384', 'zt8806_8886', 'zs9608_974101', 'zs9608_9741'])) & (orderLog['date'] < 20201215)]])
checkLog2 = pd.concat([orderLog[(orderLog['server_account'].isin(['zs6601_6623', 'zs6601_6627', 'zs6601_6683', 'zt8803_897002'])) & (orderLog['date'] >= 20201214) & (orderLog['sta'] == '3. sta300')],
                       orderLog[(orderLog['server_account'].isin(['zt5205_537501', 'zs5206_5384', 'zt8806_8886', 'zs9608_974101', 'zs9608_9741'])) & (orderLog['date'] >= 20201215) & (orderLog['sta'] == '3. sta300')]])
checkLog1['tag'] = 'before rollout'
checkLog2['tag'] = 'after rollout'
checkLog = pd.concat([checkLog1, checkLog2])
checkLog = checkLog[(checkLog["updateType"] == 0)]
checkLog['nn'] = checkLog.groupby(['exchange', 'server_account', 'secid'])['tag'].transform('nunique')
checkLog = checkLog[checkLog['nn'] == 2]
save = checkLog.groupby(['server_account', 'secid'])['date'].size().reset_index()

checkLog1 = orderLog[(orderLog['secid'].isin(checkLog['secid'].unique())) & (orderLog['updateType'] == 0)]
checkLog1['brokerNum'] = checkLog1.groupby(['date', 'secid', 'vai'])['server_account'].transform('nunique')
checkLog1['num'] = np.where(checkLog1['brokerNum'] == 1, '0', np.where(
(checkLog1['brokerNum'] >= 2) & (checkLog1['brokerNum'] <= 3), '1-2', '>2'))
c1 = pd.concat([checkLog1[(checkLog1['server_account'].isin(['zs6601_6623', 'zs6601_6627', 'zs6601_6683', 'zt8803_897002'])) & (checkLog1['date'] < 20201214)],
                       checkLog1[(checkLog1['server_account'].isin(['zt5205_537501', 'zs5206_5384', 'zt8806_8886', 'zs9608_974101', 'zs9608_9741'])) & (checkLog1['date'] < 20201215)]])
c2 = pd.concat([checkLog1[(checkLog1['server_account'].isin(['zs6601_6623', 'zs6601_6627', 'zs6601_6683', 'zt8803_897002'])) & (checkLog1['date'] >= 20201214) & (checkLog1['sta'] == '3. sta300')],
                       checkLog1[(checkLog1['server_account'].isin(['zt5205_537501', 'zs5206_5384', 'zt8806_8886', 'zs9608_974101', 'zs9608_9741'])) & (checkLog1['date'] >= 20201215) & (checkLog1['sta'] == '3. sta300')]])
c1['tag'] = 'before rollout'
c2['tag'] = 'after rollout'
checkLog1 = pd.concat([c1, c2])
checkLog1 = pd.merge(checkLog1, save[['server_account', 'secid']], on=['server_account', 'secid'])
checkLog1['index'] = np.where(checkLog1['server_account'].isin(['zt5205_537501', 'zs5206_5384']), 'IF', np.where(
checkLog1['server_account'].isin(['zt8803_897002', 'zt8806_8886']), 'IC', np.where(
checkLog1['server_account'].isin(['zs6601_6623', 'zs6601_6627', 'zs6601_6683']), 'CSI1000', 'CSIRest')))
checkLog1['broker'] = checkLog1['colo'].str[:2] + '_' + checkLog1['accCode'].astype(str).str[:2]

re = checkLog1.groupby(['index', 'broker', 'tag', 'num'])['date'].size().reset_index()
re = pd.pivot_table(re, values=['date'], columns=['num'], index=['index', 'broker', 'tag']).reset_index()
re['date', 'sum'] = re['date', '0'] + re['date', '1-2'] + re['date', '>2']
re['date', '0'] = re['date', '0'] / re['date', 'sum']
re['date', '1-2'] = re['date', '1-2'] / re['date', 'sum']
re['date', '>2'] = re['date', '>2'] / re['date', 'sum']
re['date', '0'] = re['date', '0'].apply(lambda x: '%.1f%%'%(100*x))
re['date', '1-2'] = re['date', '1-2'].apply(lambda x: '%.1f%%'%(100*x))
re['date', '>2'] = re['date', '>2'].apply(lambda x: '%.1f%%'%(100*x))
from IPython.display import display, HTML
HTML(re.groupby(['index', 'broker', 'tag'])[[('date',   '0'), ('date', '1-2'), ('date', '>2')]].first().to_html())

C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [10]:
checkLog1 = pd.concat([orderLog[(orderLog['server_account'].isin(['zs6601_6623', 'zs6601_6627', 'zs6601_6683', 'zt8803_897002'])) & (orderLog['date'] < 20201214)],
                       orderLog[(orderLog['server_account'].isin(['zt5205_537501', 'zs5206_5384', 'zt8806_8886', 'zs9608_974101', 'zs9608_9741'])) & (orderLog['date'] < 20201215)]])
checkLog2 = pd.concat([orderLog[(orderLog['server_account'].isin(['zs6601_6623', 'zs6601_6627', 'zs6601_6683', 'zt8803_897002'])) & (orderLog['date'] >= 20201214) & (orderLog['sta'] == '3. sta300')],
                       orderLog[(orderLog['server_account'].isin(['zt5205_537501', 'zs5206_5384', 'zt8806_8886', 'zs9608_974101', 'zs9608_9741'])) & (orderLog['date'] >= 20201215) & (orderLog['sta'] == '3. sta300')]])
checkLog1['tag'] = 'before rollout'
checkLog2['tag'] = 'after rollout'
checkLog = pd.concat([checkLog1, checkLog2])
checkLog['count'] = checkLog.groupby(['exchange', 'server_account', 'secid'])['tag'].transform('nunique')
checkLog = checkLog[checkLog['count'] == 2]
checkLog['index'] = np.where(checkLog['server_account'].isin(['zt5205_537501', 'zs5206_5384']), 'IF', np.where(
checkLog['server_account'].isin(['zt8803_897002', 'zt8806_8886']), 'IC', np.where(
checkLog['server_account'].isin(['zs6601_6623', 'zs6601_6627', 'zs6601_6683']), 'CSI1000', 'CSIRest')))
checkLog['broker'] = checkLog['colo'].str[:2] + '_' + checkLog['accCode'].astype(str).str[:2]

d1 = checkLog[(checkLog['updateType'] == 0)].groupby(['index', 'broker', 'tag', 'exchange'])['orderNtl'].sum().reset_index()
d2 = checkLog[(checkLog['updateType'] == 4)].groupby(['index', 'broker', 'tag', 'exchange'])['tradeNtl'].sum().reset_index()
re2 = pd.merge(d1, d2, on=['index', 'broker', 'tag', 'exchange'])
re2['fill rate'] = re2['tradeNtl'] / re2['orderNtl']
re2['fill rate'] = re2['fill rate'].apply(lambda x: '%.1f%%'%(100*x))
from IPython.display import display, HTML
HTML(re2.groupby(['exchange','index', 'broker', 'tag'])['orderNtl', 'tradeNtl', 'fill rate'].first().to_html())

In [12]:
test_data = []
for server in ['zs6601_6623', 'zs6601_6627', 'zs6601_6683', 'zt8803_897002', 'zt5205_537501', 'zs5206_5384', 'zt8806_8886', 'zs9608_974101', 'zs9608_9741']:
    cc = orderLog[orderLog['secid'].isin(checkLog[checkLog['server_account'] == server]['secid'].unique())]
    cc = cc[(~cc['server_account'].isin(['zs6601_6623', 'zs6601_6627', 'zs6601_6683', 'zt8803_897002', 'zt5205_537501', 'zs5206_5384', 'zt8806_8886', 'zs9608_974101', 'zs9608_9741']))]
    cc['tag'] = 'before rollout'
    if server in ['zs6601_6623', 'zs6601_6627', 'zs6601_6683', 'zt8803_897002']:
        date = 20201214
    else:
        date = 20201215
    cc.loc[(cc['date'] >= date), 'tag'] = 'after rollout'
    cc['num'] = cc.groupby(['exchange', 'server_account', 'secid'])['tag'].transform('nunique')
    cc = cc[cc['num'] == 2]
    if server in ['zt5205_537501', 'zs5206_5384']:
        cc['index'] = 'IF'
    elif server in ['zt8803_897002', 'zt8806_8886']:
        cc['index'] = 'IC'
    elif server in ['zs6601_6623', 'zs6601_6627', 'zs6601_6683']:
        cc['index'] = 'CSI1000'
    else:
        cc['index'] = 'CSIRest'
    cc['server'] = server
    test_data += [cc]
test_data = pd.concat(test_data)
test_data['broker'] = test_data['server'].apply(lambda x: x[:2] + '_' + x.split('_')[1][:2])
d1 = test_data[(test_data['updateType'] == 0)].groupby(['index', 'broker', 'tag', 'exchange'])['orderNtl'].sum().reset_index()
d2 = test_data[(test_data['updateType'] == 4)].groupby(['index', 'broker', 'tag', 'exchange'])['tradeNtl'].sum().reset_index()
re3 = pd.merge(d1, d2, on=['index', 'broker', 'tag', 'exchange'])
re3['fill rate'] = re3['tradeNtl'] / re3['orderNtl']
re3['fill rate'] = re3['fill rate'].apply(lambda x: '%.1f%%'%(100*x))
from IPython.display import display, HTML
HTML(re3.groupby(['exchange','index', 'broker', 'tag'])['orderNtl', 'tradeNtl', 'fill rate'].first().to_html())

In [13]:
re3 = re3.rename(columns={'fill rate':'fill rate in other accounts'})
re = pd.merge(re2, re3[['exchange', 'index', 'broker', 'tag', 'fill rate in other accounts']], on=['exchange', 'index', 'broker', 'tag'])
HTML(re.groupby(['exchange','index', 'broker', 'tag'])['orderNtl', 'tradeNtl', 'fill rate', 'fill rate in other accounts'].first().to_html())

In [20]:
orderLog['sdd'] = orderLog.groupby(['order'])['sdd'].transform('first')
df = orderLog[(orderLog['updateType'] == 4) & (orderLog['date'] >= 20201214)]
df = df.groupby('order').last().reset_index()

if df[df['absFilledThisUpdate'] == 0].shape[0] != 0:
    print('There are stocks with zero trade size')
    print(df[df['absFilledThisUpdate'] == 0].groupby(['colo', 'accCode'])['secid'].size())
    df = df[df['absFilledThisUpdate'] != 0]
if df[df['beta_60'].isnull()].shape[0] != 0:
    print('There are stocks with null beta')
    print(df[df['beta_60'].isnull()])
    df = df[~df['beta_60'].isnull()]
df['max_trade'] = df.groupby('order')['absOrderSizeCumFilled'].transform('max')
df['last'] = 0
df.loc[df['max_trade'] == df['absOrderSizeCumFilled'], 'last'] = 1

df["buyRet"] = np.where(df["orderDirection"].isin([1, 2]), df["adjMid_F90s"] / df["tradePrice"] - 1, np.nan)
df["buyRet1"] = np.where(df["orderDirection"].isin([1, 2]), df["adjMid_F300s"] / df["tradePrice"] - 1, np.nan)
df["sellRet"] = np.where(df["orderDirection"].isin([-1, -2]), df["tradePrice"] / df["adjMid_F90s"] - 1, np.nan)
df["sellRet1"] = np.where(df["orderDirection"].isin([-1, -2]), df["tradePrice"] / df["adjMid_F300s"] - 1, np.nan)
df["indexRet"] = df["indexClose_F90s"]/df["indexClose"] - 1
df["indexRet1"] = df["indexClose_F300s"]/df["indexClose"] - 1
df["buyAlpha"] = np.where(df["orderDirection"].isin([1, 2]), df["buyRet"] - df["indexRet"] * df["beta_60"], np.nan)
df["buyAlpha1"] = np.where(df["orderDirection"].isin([1, 2]), df["buyRet1"] - df["indexRet1"] * df["beta_60"], np.nan)
df["sellAlpha"] = np.where(df["orderDirection"].isin([-1, -2]), df["sellRet"] + df["indexRet"] * df["beta_60"], np.nan)
df["sellAlpha1"] = np.where(df["orderDirection"].isin([-1, -2]), df["sellRet1"] + df["indexRet1"] * df["beta_60"], np.nan)
df["buyNum"] = np.where((df["orderDirection"].isin([1, 2])) & (df['last'] == 1), 1, 0)
df["sellNum"] = np.where((df["orderDirection"].isin([-1, -2])) & (df['last'] == 1), 1, 0)
df['server_account'] = df['colo'].apply(lambda x: x.replace('_', '')) + '_' + df['accCode'].astype(str)

print('2. Orders with nan ars in updateType 0')
if df[df['ars'].isnull()].shape[0] != 0:
    print(df[df['ars'].isnull()])
    df = df[~df['ars'].isnull()]
df.loc[(df['date'] <= 20201013) & (df['ars'].isin([121, 131])), 'mrstauc'] = 1000
df.loc[(df['date'] <= 20201013) & (df['ars'].isin([221, 231])), 'mrstauc'] = 2000
df.loc[(df['date'] <= 20201013) & (df['ars'].isin([321, 331])), 'mrstauc'] = 3000
df.loc[(df['date'] <= 20201013) & (~df['ars'].isin([121, 221, 321, 131, 231, 331])), 'mrstauc'] = 0
df.loc[(df['date'] <= 20201013) & (df['mrstauc'].isin([1000, 2000, 3000])), 'mrstaat'] = 3000
df.loc[(df['date'] <= 20201013) & (df['mrstauc'] == 0), 'mrstaat'] = 1000

print('3. Current strategy status')
if 2 in df['mrstauc'].unique():
    print(df[df['mrstauc'] == 2].groupby('mrstaat')['mrstauc'].unique())
    df.loc[df['mrstauc'] == 2, 'mrstauc'] = 0
print(df.groupby('mrstaat')['mrstauc'].unique())

print('4. Strange strategy')
if df[(df["mrstaat"].isnull()) | (df['mrstaat'] == 0) | (df['mrstaat'] == 2000)].shape[0] != 0:
    print('1) null mrstaat value')
    print(df[(df["mrstaat"].isnull())][['date', 'secid', 'vai', 'sdd', 'absFilledThisUpdate', 'colo', 'accCode', 'mrstaat', 'mrstauc', 'ars']])
    print(df[(df["mrstaat"].isnull())].groupby(['date', 'colo', 'accCode'])['secid'].size())
    print('2) 0 mrstaat value')
    print(df[(df["mrstaat"] == 0)][['date', 'secid', 'vai', 'sdd', 'absFilledThisUpdate', 'colo', 'accCode', 'mrstaat', 'mrstauc', 'ars']])
    print('3) 2000 mrstaat value')
    print(df[(df["mrstaat"] == 2000)][['date', 'secid', 'vai', 'sdd', 'absFilledThisUpdate', 'colo', 'accCode', 'mrstaat', 'mrstauc', 'ars']])
df = df[(~df["mrstaat"].isnull()) & (df['mrstaat'] != 0) & (df['mrstaat'] != 2000)]

if 'mrsb90' in df.columns:
    df['m1'] = df['mrstaat'].apply(lambda x: x - (x // 10000) * 10000)
    df['m2'] = df['mrstauc'].apply(lambda x: x - (x // 10000) * 10000)
    if df[df['mrsb90'] == '-'].shape[0] != 0:
        print('There are ticks with mrsb90 value -')
        print(df[df['mrsb90'] == '-'][['date', 'secid', 'vai', 'sdd', 'absFilledThisUpdate', 'colo', 'accCode', 'mrstaat', 'mrstauc', 'aaa', 'ars']])
        df = df[df['mrsb90'] != '-']
    if df[df['aaa'] == '-'].shape[0] != 0:
        print('There are ticks with aaa value -')
        print(df[df['aaa'] == '-'][['date', 'secid', 'vai', 'sdd', 'absFilledThisUpdate', 'colo', 'accCode', 'mrstaat', 'mrstauc', 'aaa', 'ars']])
        df = df[df['aaa'] != '-']   
    df['mrsb90'] = df['mrsb90'].astype(float)
    df['aaa'] = df['aaa'].astype(float)
    df.loc[(df['orderDirection'] >= 1) &\
             (df['mrstaat'].isin([11000, 13000])) & (abs(df['aaa'] - df['mrsb90']) < 1e-12), 'mrstauc'] = \
    df.loc[(df['orderDirection'] >= 1) &\
             (df['mrstaat'].isin([11000, 13000])) & (abs(df['aaa'] - df['mrsb90']) < 1e-12), 'm2']

    df.loc[(df['orderDirection'] >= 1) &\
             (df['mrstaat'].isin([11000, 13000])) & (abs(df['aaa'] - df['mrsb90']) < 1e-12), 'mrstaat'] = \
    df.loc[(df['orderDirection'] >= 1) &\
             (df['mrstaat'].isin([11000, 13000])) & (abs(df['aaa'] - df['mrsb90']) < 1e-12), 'm1']

    df.loc[(df['orderDirection'] < 1) &\
             (df['mrstaat'].isin([11000, 13000])) & (abs(df['aaa'] - df['mrss90']) < 1e-12), 'mrstauc'] = \
    df.loc[(df['orderDirection'] < 1) &\
             (df['mrstaat'].isin([11000, 13000])) & (abs(df['aaa'] - df['mrss90']) < 1e-12), 'm2']

    df.loc[(df['orderDirection'] < 1) &\
             (df['mrstaat'].isin([11000, 13000])) & (abs(df['aaa'] - df['mrss90']) < 1e-12), 'mrstaat'] = \
    df.loc[(df['orderDirection'] < 1) &\
             (df['mrstaat'].isin([11000, 13000])) & (abs(df['aaa'] - df['mrss90']) < 1e-12), 'm1']       
print(df['mrstaat'].unique())
df['sta'] = np.where(df['mrstaat'] == 1000, '1. staone', np.where(
df['mrstaat'] == 3000, '2. statwo', '3. sta300'))
df.loc[df['mrstauc'] == 2, 'mrstauc'] = 0    
df['exchange'] = np.where(df['secid'] >= 2000000, 'SZE', 'SSE')

df = df[((df['sdd'] > 94000000) & (df['sdd'] < 130000000)) | (df['sdd'] > 131000000)]
df = df[df['sta'].isin(['2. statwo', '3. sta300'])]
df['broker'] = df['colo'].str[:2] + '_' + df['accCode'].astype(str).str[:2]
re = []

for server in ['zs6601_6623', 'zs6601_6627', 'zs6601_6683', 'zt8803_897002', 'zt5205_537501', 'zs5206_5384', 'zt8806_8886', 'zs9608_974101', 'zs9608_9741']:  
    df1 = df.copy()
    df1['tag'] = np.where((df1['server_account'] == server) & (df1['sta'] == '3. sta300'), 1, np.where(
    (~df1['server_account'].isin(['zs6601_6623', 'zs6601_6627', 'zs6601_6683', 'zt8803_897002', 'zt5205_537501', 'zs5206_5384', 'zt8806_8886', 'zs9608_974101', 'zs9608_9741'])) & (df1['sta'] == '2. statwo'), 0, 'else'))
    df1 = df1[df1['tag'] != 'else']
    df1['num'] = df1.groupby(['date', 'secid'])['tag'].transform('nunique')
    df1['tradeNtl'] = df1['tradePrice']*df1['absFilledThisUpdate']
    df1 = df1[df1['num'] == 2]
    if server in ['zt5205_537501', 'zs5206_5384']:
        df1['index'] = 'IF'
    elif server in ['zt8803_897002', 'zt8806_8886']:
        df1['index'] = 'IC'
    elif server in ['zs6601_6623', 'zs6601_6627', 'zs6601_6683']:
        df1['index'] = 'CSI1000'
    else:
        df1['index'] = 'CSIRest'
    df1['server_account'] = server
    df1['tag'] = 'sta300 account'
    df1.loc[df1['sta'] == '2. statwo', 'tag'] = 'statwo account'

    df1["buyNtl"] = np.where(~df1["buyRet"].isnull(), df1["tradeNtl"], np.nan)
    df1["sellNtl"] = np.where(~df1["sellRet"].isnull(), df1["tradeNtl"], np.nan)
    df1["sumbuyNtl"] = df1.groupby(["exchange", "index", "server_account", "tag", "sta"])["buyNtl"].transform(sum)
    df1["sumsellNtl"] = df1.groupby(["exchange", "index", "server_account", "tag", "sta"])["sellNtl"].transform(sum)

    df1["sumsellRet"] = df1["tradeNtl"] * df1["sellRet"]
    df1["sumsellRet"] = df1.groupby(["exchange", "index", "server_account", "tag", "sta"])["sumsellRet"].transform(sum)

    df1["sumbuyAlpha"] = df1["tradeNtl"] * df1["buyAlpha"]
    df1["sumbuyAlpha"] = df1.groupby(["exchange", "index", "server_account", "tag", "sta"])["sumbuyAlpha"].transform(sum)

    df1["sumsellAlpha"] = df1["tradeNtl"] * df1["sellAlpha"]
    df1["sumsellAlpha"] = df1.groupby(["exchange", "index", "server_account", "tag", "sta"])["sumsellAlpha"].transform(sum)

    df1["sumbuyRet"] = df1["tradeNtl"] * df1["buyRet"]
    df1["sumbuyRet"] = df1.groupby(["exchange", "index", "server_account", "tag", "sta"])["sumbuyRet"].transform(sum)

    df1["buyRet"] = df1["sumbuyRet"] / df1["sumbuyNtl"]
    df1["sellRet"] = df1["sumsellRet"] / df1["sumsellNtl"]
    df1["buyAlpha"] = df1["sumbuyAlpha"] / df1["sumbuyNtl"]
    df1["sellAlpha"] = df1["sumsellAlpha"] / df1["sumsellNtl"]
    df1["buyOrderNum"] = df1.groupby(["exchange", "index", "server_account", "tag", "sta"])["buyNum"].transform(sum)
    df1["sellOrderNum"] = df1.groupby(["exchange", "index", "server_account", "tag", "sta"])["sellNum"].transform(sum)
    re += [df1.groupby(["exchange", "index", "server_account", "tag", "sta"])["buyOrderNum", "buyRet", "buyAlpha", "sellOrderNum", "sellRet", "sellAlpha"].first().reset_index()]
    
re = pd.concat(re)

from IPython.display import display, HTML
for col in ["buyRet", "sellRet", "buyAlpha", "sellAlpha"]:
    re[col] = re[col].apply(lambda x: '%.2f'%(x*10000))
        
HTML(re.groupby(["exchange", "index", "server_account", "tag"])["buyOrderNum", "buyRet", "buyAlpha", "sellOrderNum", "sellRet", "sellAlpha"].first().to_html())

2. Orders with nan ars in updateType 0
3. Current strategy status
mrstaat
1000.0                                                 [0.0]
3000.0                              [1000.0, 2000.0, 3000.0]
11000.0                          [10000.0, 30000.0, 20000.0]
13000.0    [11000.0, 33000.0, 22000.0, 21000.0, 12000.0, ...
Name: mrstauc, dtype: object
4. Strange strategy
[ 3000.  1000. 13000. 11000.]


In [ ]:
re2 = orderLog[(orderLog['updateType'] == 4)].groupby(['date', 'server_account'])['tradeNtl'].sum().reset_index()
re3 = orderLog[orderLog['updateType'] == 4]
re3 = pd.merge(re3, orderLog[(orderLog['updateType'] == 4) & (orderLog['server_account'] == 'zs6601_6623')]\
         .drop_duplicates(['date', 'secid'])[['date', 'secid']], on=['date', 'secid'], how='inner')
re3 = re3[re3['server_account'] != 'zs6601_6623']
re3 = re3.groupby(['date'])['tradeNtl'].sum().reset_index()


re4 = orderLog[orderLog['updateType'] == 4]
re4 = pd.merge(re4, orderLog[(orderLog['updateType'] == 4) & (orderLog['server_account'] == 'zs6601_6627')]\
         .drop_duplicates(['date', 'secid'])[['date', 'secid']], on=['date', 'secid'], how='inner')
re4 = re4[re4['server_account'] != 'zs6601_6627']
re4 = re4.groupby(['date'])['tradeNtl'].sum().reset_index()

re5 = orderLog[orderLog['updateType'] == 4]
re5 = pd.merge(re5, orderLog[(orderLog['updateType'] == 4) & (orderLog['server_account'] == 'zs6601_6683')]\
         .drop_duplicates(['date', 'secid'])[['date', 'secid']], on=['date', 'secid'], how='inner')
re5 = re5[re5['server_account'] != 'zs6601_6683']
re5 = re5.groupby(['date'])['tradeNtl'].sum().reset_index()

re6 = orderLog[orderLog['updateType'] == 4]
re6 = pd.merge(re6, orderLog[(orderLog['updateType'] == 4) & (orderLog['server_account'] == 'zt8803_897002')]\
         .drop_duplicates(['date', 'secid'])[['date', 'secid']], on=['date', 'secid'], how='inner')
re6 = re6[re6['server_account'] != 'zt8803_897002']
re6 = re6.groupby(['date'])['tradeNtl'].sum().reset_index()



fig = plt.figure(figsize=(20, 12))
ax = fig.add_subplot(221)

ax.plot(re2[re2['server_account'] == 'zs6601_6623']['tradeNtl'].values/re2[re2['server_account'] == 'zs6601_6623']['tradeNtl'].values[0], label='zs9606_9765 Turnover')
ax.plot(re3['tradeNtl'].values/re3['tradeNtl'].values[0], label='other accounts Turnover')
indexLs = [i for i in range(re2[re2['server_account'] == 'zs6601_6623'].shape[0])]
dateLs = [int(i) for i in re2[re2['server_account'] == 'zs6601_6623']['date'].values]
ax.set_xticks(indexLs)
ax.set_xticklabels(dateLs)
ax.legend()
ax.tick_params(labelsize=10)
ax.set_title('Turnover of stocks trade in zs6601_6623')
plt.xticks(rotation=60)
plt.axvline(x=5,ls="--",c="grey", alpha=0.2, lw=1)


ax = fig.add_subplot(222)


ax.plot((list(re2[re2['server_account'].isin(['zs6601_6627'])]['tradeNtl'].values))/re2[re2['server_account'].isin(['zs6601_6627'])]['tradeNtl'].values[0], label='zs6601_6627 Turnover')
ax.plot((list(re4['tradeNtl'].values))/re4['tradeNtl'].values[0], label='other accounts Turnover')
indexLs = [i for i in range(re2[re2['server_account'] == 'zs6601_6627'].shape[0])]
dateLs = [int(i) for i in re2[re2['server_account'] == 'zs6601_6627']['date'].values]
ax.set_xticks(indexLs)
ax.set_xticklabels(dateLs)
ax.legend()
ax.tick_params(labelsize=10)
ax.set_title('Turnover of stocks trade in zs6601_6627')
plt.xticks(rotation=60)
plt.axvline(x=5,ls="--",c="grey", alpha=0.2, lw=1)

ax = fig.add_subplot(223)


ax.plot((list(re2[re2['server_account'].isin(['zs6601_6683'])]['tradeNtl'].values))/re2[re2['server_account'].isin(['zs6601_6683'])]['tradeNtl'].values[0], label='zs6601_6683 Turnover')
ax.plot((list(re5['tradeNtl'].values))/re5['tradeNtl'].values[0], label='other accounts Turnover')
indexLs = [i for i in range(re2[re2['server_account'] == 'zs6601_6683'].shape[0])]
dateLs = [int(i) for i in re2[re2['server_account'] == 'zs6601_6683']['date'].values]
ax.set_xticks(indexLs)
ax.set_xticklabels(dateLs)
ax.legend()
ax.tick_params(labelsize=10)
ax.set_title('Turnover of stocks trade in zs6601_6683')
plt.xticks(rotation=60)
plt.axvline(x=5,ls="--",c="grey", alpha=0.2, lw=1)

ax = fig.add_subplot(224)


ax.plot((list(re2[re2['server_account'].isin(['zt8803_897002'])]['tradeNtl'].values))/re2[re2['server_account'].isin(['zt8803_897002'])]['tradeNtl'].values[0], label='zt8803_897002 Turnover')
ax.plot((list(re6['tradeNtl'].values))/re6['tradeNtl'].values[0], label='other accounts Turnover')
indexLs = [i for i in range(re2[re2['server_account'] == 'zt8803_897002'].shape[0])]
dateLs = [int(i) for i in re2[re2['server_account'] == 'zt8803_897002']['date'].values]
ax.set_xticks(indexLs)
ax.set_xticklabels(dateLs)
ax.legend()
ax.tick_params(labelsize=10)
ax.set_title('Turnover of stocks trade in zt8803_897002')
plt.xticks(rotation=60)
plt.axvline(x=5, ls="--",c="grey", alpha=0.2, lw=1)
plt.tight_layout()
plt.show()